<a href="https://colab.research.google.com/github/Viplove12/sentiment-analysis/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [0]:

dataset=pd.read_csv('/content/drive/My Drive/sentiment analysis (1)/train.csv',encoding='latin-1')

In [195]:
len(dataset)

99989

In [181]:

import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus=[]
print("done")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
done


## **Data PreProcessing**

In [182]:
for i in range(0,len(dataset)):
    review=re.sub('[^a-zA-Z]',' ',dataset['SentimentText'][i])
    review=review.lower()
    review=review.split()
    ps=PorterStemmer()
    review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review=' '.join(review)
    corpus.append(review)

all_text=' '.join(corpus)

words=all_text.split()
print('done')

done


In [199]:
len(words)

782808

## Encoding (because pytorch take encoded values only)

In [200]:
from collections import Counter
counts=Counter(words)
vocab=sorted(counts,key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
reviews_ints = []
for review in corpus:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])
print('done')

done


In [202]:
len (reviews_ints)

99989

## Removing 0 lenght tweets

In [203]:


    
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))
sum=0;

#FOR SEQUENCE LENGTH
for x in reviews_ints:
    sum+=len(x);

sum=sum/len(reviews_ints)   

#For finding the max length
lenght=0
size=len(corpus)
for i in range(size):
    if lenght<len(corpus[i]):
        lenght=len(corpus[i])

print('Number of reviews before removing outliers: ', len(reviews_ints))

## remove any reviews/labels with zero length from the reviews_ints list.
# get indices of any reviews with length 0
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]
print(len(non_zero_idx))

# remove 0-length reviews and their labels
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
new_labels = np.array([dataset['Sentiment'][ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(reviews_ints))


Zero-length reviews: 32
Maximum review length: 79
Number of reviews before removing outliers:  99989
99957
Number of reviews after removing outliers:  99957


## Padding the Sequence

In [0]:

def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features    

seq_length = 150

features = pad_features(reviews_ints, seq_length=seq_length)


##Spliting the dataset

In [205]:
from sklearn.model_selection import train_test_split
xtrain,xtra,ytrain,ytra=train_test_split(features,new_labels,test_size=0.2,random_state=0)    
xval,xtest,yval,ytest=train_test_split(xtra,ytra,test_size=0.5,random_state=0)    

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(xtrain.shape), 
      "\nValidation set: \t{}".format(xval.shape),
      "\nTest set: \t\t{}".format(xtest.shape)
     
     )



			Feature Shapes:
Train set: 		(79965, 150) 
Validation set: 	(9996, 150) 
Test set: 		(9996, 150)


In [191]:
ytest.shape



(3195,)

##Dataloading

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(xtrain), torch.from_numpy(ytrain))
valid_data = TensorDataset(torch.from_numpy(xval), torch.from_numpy(yval))
test_data = TensorDataset(torch.from_numpy(xtest), torch.from_numpy(ytest))

# dataloaders
batch_size = 32

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size,drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size,drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size,drop_last=True)



In [207]:
print(len(train_loader))
print(len(valid_loader))
print(len(test_loader))

2498
312
312


##Checking for GPU

In [208]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')


Training on GPU.


#Creating the architecture of RNN(using class)

In [0]:

import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc1 = nn.Linear(hidden_dim, 100)
        self.fc2 = nn.Linear(100, 1)
        
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc1(out)
        out=self.fc2(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
         # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
         # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:    
          hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

##Model Parameters

In [269]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 512
n_layers = 3

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
print('done')

done


##loss and optimization functions

In [270]:

lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
print('done')

done


#Training

In [271]:
# training params

epochs = 2# 3 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))


Epoch: 1/2... Step: 100... Loss: 0.557674... Val Loss: 0.621793
Epoch: 1/2... Step: 200... Loss: 0.615410... Val Loss: 0.597965
Epoch: 1/2... Step: 300... Loss: 0.508855... Val Loss: 0.586846
Epoch: 1/2... Step: 400... Loss: 0.525422... Val Loss: 0.579808
Epoch: 1/2... Step: 500... Loss: 0.595365... Val Loss: 0.580435
Epoch: 1/2... Step: 600... Loss: 0.509907... Val Loss: 0.556046
Epoch: 1/2... Step: 700... Loss: 0.502599... Val Loss: 0.555449
Epoch: 1/2... Step: 800... Loss: 0.587867... Val Loss: 0.564071
Epoch: 1/2... Step: 900... Loss: 0.522794... Val Loss: 0.549660
Epoch: 1/2... Step: 1000... Loss: 0.623706... Val Loss: 0.547534
Epoch: 1/2... Step: 1100... Loss: 0.632756... Val Loss: 0.532119
Epoch: 1/2... Step: 1200... Loss: 0.571723... Val Loss: 0.539207
Epoch: 1/2... Step: 1300... Loss: 0.581109... Val Loss: 0.529637
Epoch: 1/2... Step: 1400... Loss: 0.583958... Val Loss: 0.530410
Epoch: 1/2... Step: 1500... Loss: 0.470295... Val Loss: 0.527084
Epoch: 1/2... Step: 1600... Loss: 

#Testing

In [272]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
ypred=[]
count=0

for inputs, labels in test_loader:
    count+=1
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    ##ypred.append()
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    #print(len(pred),len(labels))
    ypred.extend(pred)
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

#print(len(ypred))
ytestt=ytest[:len(ypred)]
#print(len(ypred))
from sklearn.metrics import f1_score
test=np.asarray(ytestt)
ypred=np.asarray(ypred)
ypred=ypred.astype(int)
score=f1_score(ytestt,ypred)
print("F1 score is: {:.3f}".format(score))

from sklearn.metrics import average_precision_score
average_precision = average_precision_score(ytestt,ypred)
print('Average precision-recall score: {:0.3f}'.format(average_precision))
from sklearn.metrics import recall_score
recall_score=recall_score(ytestt, ypred)
print('Average recall score: {:0.3f}'.format(recall_score))


Test loss: 0.529
Test accuracy: 0.733
F1 score is: 0.554
Average precision-recall score: 0.560
Average recall score: 0.546


In [0]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

cnf_matrix = confusion_matrix(ytestt, ypred)



In [235]:
cnf_matrix

array([[1865, 2535],
       [2340, 3244]])